<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

# Embeddings

Привет! В этом домашнем задании мы с помощью эмбеддингов решим задачу семантической классификации твитов.

Для этого мы воспользуемся предобученными эмбеддингами word2vec.

Для начала скачаем датасет для семантической классификации твитов:

In [224]:
!gdown https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph&export=download
!unzip archive.zip

Downloading...
From: https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph
To: /content/archive.zip
84.9MB [00:00, 111MB/s] 
Archive:  archive.zip
replace training.1600000.processed.noemoticon.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


Импортируем нужные библиотеки:

In [2]:
import math
import random
import string

import numpy as np
import pandas as pd
import seaborn as sns

import torch
import nltk
import gensim
import gensim.downloader as api

In [3]:
random.seed(42)
np.random.seed(42)
torch.random.manual_seed(42)
torch.cuda.random.manual_seed(42)
torch.cuda.random.manual_seed_all(42)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="latin", header=None, names=["emotion", "id", "date", "flag", "user", "text"])

Посмотрим на данные:

In [5]:
data.head()

,emotion,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


Выведем несколько примеров твитов, чтобы понимать, с чем мы имеем дело:

In [6]:
examples = data["text"].sample(10)
print("\n".join(examples))

@chrishasboobs AHHH I HOPE YOUR OK!!! 
@misstoriblack cool , i have no tweet apps  for my razr 2
@TiannaChaos i know  just family drama. its lame.hey next time u hang out with kim n u guys like have a sleepover or whatever, ill call u
School email won't open  and I have geography stuff on there to revise! *Stupid School* :'(
upper airways problem 
Going to miss Pastor's sermon on Faith... 
on lunch....dj should come eat with me 
@piginthepoke oh why are you feeling like that? 
gahh noo!peyton needs to live!this is horrible 
@mrstessyman thank you glad you like it! There is a product review bit on the site  Enjoy knitting it!


Как вилим, тексты твитов очень "грязные". Нужно предобработать датасет, прежде чем строить для него модель классификации.

Чтобы сравнивать различные методы обработки текста/модели/прочее, разделим датасет на dev(для обучения модели) и test(для получения качества модели).

In [7]:
indexes = np.arange(data.shape[0])
np.random.shuffle(indexes)
dev_size = math.ceil(data.shape[0] * 0.8)

dev_indexes = indexes[:dev_size]
test_indexes = indexes[dev_size:]

dev_data = data.iloc[dev_indexes]
test_data = data.iloc[test_indexes]

dev_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

## Обработка текста

Токенизируем текст, избавимся от знаков пунктуации и выкинем все слова, состоящие менее чем из 4 букв:

In [8]:
tokenizer = nltk.WordPunctTokenizer()
line = tokenizer.tokenize(dev_data["text"][0].lower())
print(" ".join(line))

@ claire_nelson i ' m on the north devon coast the next few weeks will be down in devon again in may sometime i hope though !


In [9]:
filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
print(" ".join(filtered_line))

north devon coast next weeks will down devon again sometime hope though


In [10]:
filtered_line

['north',
 'devon',
 'coast',
 'next',
 'weeks',
 'will',
 'down',
 'devon',
 'again',
 'sometime',
 'hope',
 'though']

Загрузим предобученную модель эмбеддингов. 

Если хотите, можно попробовать другую. Полный список можно найти здесь: https://github.com/RaRe-Technologies/gensim-data.

Данная модель выдает эмбеддинги для **слов**. Строить по эмбеддингам слов эмбеддинги предложений мы будем ниже.

In [11]:
word2vec = api.load("word2vec-google-news-300")

In [12]:
emb_line = [word2vec.get_vector(w) for w in filtered_line if w in word2vec]
print(sum(emb_line).shape)

(300,)


Нормализуем эмбеддинги, прежде чем обучать на них сеть. 
(наверное, вы помните, что нейронные сети гораздо лучше обучаются на нормализованных данных)

In [13]:
mean = np.mean(word2vec.vectors, 0)
std = np.std(word2vec.vectors, 0)
norm_emb_line = [(word2vec.get_vector(w) - mean) / std for w in filtered_line if w in word2vec and len(w) > 3]
print(sum(norm_emb_line).shape)
print([all(norm_emb_line[i] == emb_line[i]) for i in range(len(emb_line))])

(300,)
[False, False, False, False, False, False, False, False, False, False, False, False]


In [14]:
norm_emb_line[0][:5], emb_line[0][:5]

(array([-0.36991876, -1.1144205 ,  0.33723447, -0.9588026 ,  1.1273683 ],
       dtype=float32),
 array([-0.04541016, -0.13769531,  0.03027344, -0.09277344,  0.13867188],
       dtype=float32))

Сделаем датасет, который будет по запросу возвращать подготовленные данные.

In [41]:
from torch.utils.data import Dataset, random_split


class TwitterDataset(Dataset):  
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        tokens = self.tokenizer.tokenize(text)
        tokens = [w for w in tokens if all(c not in string.punctuation for c in w) and len(w) > 3]
        tokens = [w for w in tokens if w in word2vec]
        return tokens

    def get_embeddings_(self, tokens):
        # Получи эмбеддинги слов и усредни их
        embeddings = [(self.word2vec.get_vector(token) - self.mean) / self.std for token in (tokens)]
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [16]:
a = [word2vec.get_vector(token).shape for token in ["cat"]]
np.array(a).reshape(-1, 1)

array([[300]])

In [42]:
dev = TwitterDataset(dev_data, "text", "emotion", word2vec)

In [ ]:
for data in dev:
    print(data["feature"])
    break

Отлично, мы готовы с помощью эмбеддингов слов превращать твиты в векторы и обучать нейронную сеть.

Превращать твиты в векторы, используя эмбеддинги слов, можно несколькими способами. А именно такими:

## Average embedding (2 балла)
---
Это самый простой вариант, как получить вектор предложения, используя векторные представления слов в предложении. А именно: вектор предложения есть средний вектор всех слов в предлоежнии (которые остались после токенизации и удаления коротких слов, конечно). 

In [18]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

examples = {"features": [np.sum(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

1280


In [ ]:
examples["features"]

Давайте сделаем визуализацию полученных векторов твитов тренировочного (dev) датасета. Так мы увидим, насколько хорошо твиты с разными target значениями отделяются друг от друга, т.е. насколько хорошо усреднение эмбеддингов слов предложения передает информацию о предложении.

Для визуализации векторов надо получить их проекцию на плоскость. Сделаем это с помощью `PCA`. Если хотите, можете вместо PCA использовать TSNE: так у вас получится более точная проекция на плоскость (а значит, более информативная, т.е. отражающая реальное положение векторов твитов в пространстве). Но TSNE будет работать намного дольше.

In [20]:
from sklearn.decomposition import PCA


pca = PCA(n_components=2)
examples["transformed_features"] = pca.fit_transform(examples["features"])

In [21]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [24]:
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Figure(id='1232', ...)

Скорее всего, на визуализации нет четкого разделения твитов между классами. Это значит, что по полученным нами векторам твитов не так-то просто определить, к какому классу твит пренадлежит. Значит, обычный линейный классификатор не очень хорошо справится с задачей. Надо будет делать глубокую (хотя бы два слоя) нейронную сеть.

Подготовим загрузчики данных.
Усреднее векторов будем делать в "батчевалке"(`collate_fn`). Она используется для того, чтобы собирать из данных `torch.Tensor` батчи, которые можно отправлять в модель.


In [43]:
from torch.utils.data import DataLoader


batch_size = 1024
num_workers = 4

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
for data in train_loader:
    print(data['features'].shape)
    break

Определим функции для тренировки и теста модели:

In [44]:
from tqdm.notebook import tqdm


def training(model, optimizer, criterion, train_loader, epoch, device="cpu"):
    pbar = tqdm(train_loader, desc=f"Epoch {e + 1}. Train Loss: {0}")
    model.train()
    for batch in pbar:
        features = batch["features"].to(device)
        targets = batch["targets"].to(device)

        optimizer.zero_grad()
        # Получи предсказания модели
        preds = model.forward(features)
        loss = criterion(preds, targets) # Посчитай лосс
        # Обнови параметры модели
        loss.backward()
        optimizer.step()

        pbar.set_description(f"Epoch {e + 1}. Train Loss: {loss:.4}")
    

def testing(model, criterion, test_loader, device="cpu"):
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch["features"].to(device)
            targets = batch["targets"].to(device)

            # Получи предсказания модели
            preds = model.forward(features)
            #print(torch.max(preds, 1)[1])
            #print(torch.max(preds, dim=1))
            loss = criterion(preds, targets)
            #print(preds) # Посчитай лосс
            acc = sum(torch.argmax(preds, dim=1) == targets) # Посчитай точность модели

            mean_loss += loss.item()
            #print(acc.item(), acc.item() / len(targets))
            mean_acc += acc.item() / len(targets)

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc.item() / len(targets):.4}")

    pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, Test Acc: {mean_acc / len(test_loader):.4}")

    return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader)}

Создадим модель, оптимизатор и целевую функцию. Вы можете сами выбрать количество слоев в нейронной сети, ваш любимый оптимизатор и целевую функцию.


In [45]:
import torch.nn as nn
class Classifier(nn.Module):
    def __init__(self, input_dim, hidden, num_classes):
        super(Classifier, self).__init__()
        self.linear1 = nn.Linear(input_dim, hidden)
        self.linear2 = nn.Linear(hidden, hidden // 2)
        self.linear3 = nn.Linear(hidden // 2, num_classes)
        self.dropout = nn.Dropout(p=0.5)
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, x):
        out = self.linear1(x)
        out = self.relu(self.dropout(out))
        out = self.linear2(out)
        out = self.relu(self.dropout(out))
        return self.linear3(out)

In [46]:
import torch.nn as nn
from torch.optim import Adam


# Не забудь поиграться с параметрами ;)
vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 1e-3
num_epochs = 5

model = Classifier(300, 150, num_classes)# Твоя модель
model = model.to(device)
criterion = nn.CrossEntropyLoss() # Твой лосс
optimizer = torch.optim.Adam(model.parameters(), lr=lr) # Твой оптимайзер

Наконец, обучим модель и протестируем её.

После каждой эпохи будем проверять качество модели на валидационной части датасета. Если метрика стала лучше, будем сохранять модель. **Подумайте, какая метрика (точность или лосс) будет лучше работать в этой задаче?** 

In [47]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



{'Test Loss': 0.5215935312509536, 'Test Acc': 0.73997265625}



{'Test Loss': 0.513229947924614, 'Test Acc': 0.74503515625}



{'Test Loss': 0.5108161576986313, 'Test Acc': 0.747046875}



{'Test Loss': 0.5083011928796768, 'Test Acc': 0.74924609375}



{'Test Loss': 0.5067585996389389, 'Test Acc': 0.74973828125}


In [48]:
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



{'Test Loss': 0.5073666849646705, 'Test Acc': 0.7506115215654952}


## Embeddings for unknown words (8 баллов)

Пока что использовалась не вся информация из текста. Часть информации фильтровалось – если слова не было в словаре эмбеддингов, то мы просто превращали слово в нулевой вектор. Хочется использовать информацию по-максимуму. Поэтому рассмотрим другие способы обработки слов, которых нет в словаре. А именно:

- Для каждого незнакомого слова будем запоминать его контекст(слова слева и справа от этого слова). Эмбеддингом нашего незнакомого слова будет сумма эмбеддингов всех слов из его контекста. (4 балла)
- Для каждого слова текста получим его эмбеддинг из Tfidf с помощью ```TfidfVectorizer``` из [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer). Итоговым эмбеддингом для каждого слова будет сумма двух эмбеддингов: предобученного и Tfidf-ного. Для слов, которых нет в словаре предобученных эмбеддингов, результирующий эмбеддинг будет просто полученный из Tfidf. (4 балла)

Реализуйте оба варианта **ниже**. Напишите, какой способ сработал лучше и ваши мысли, почему так получилось.

In [49]:
class TwitterDatasetAverage(Dataset):  
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        tokens = self.tokenizer.tokenize(text)
        tokens = [w for w in tokens if all(c not in string.punctuation for c in w) and len(w) > 3]
        return tokens

    def get_embeddings_(self, tokens):
        # Получи эмбеддинги слов и усредни их
        embeddings = []
        window_size = 1
        for idx, token in enumerate(tokens):
            if token in self.word2vec:
                embeddings.append((self.word2vec.get_vector(token) - self.mean) / self.std)
            else:
                start_idx = max(0, idx - window_size)
                end_idx = min(len(tokens), idx + window_size)
                pos_in_window = idx
                if idx - pos_in_window < 0:
                    pos_in_window += idx - window_size
                collector = np.zeros((1, self.word2vec.vector_size)).flatten()
                flag_1 = True
                flag_2 = True
                cnt = 0
                while flag_1:
                    if tokens[start_idx] in word2vec and start_idx > 0 and start_idx != pos_in_window:
                        collector += (self.word2vec.get_vector(tokens[start_idx]) - self.mean) / self.std 
                        flag_1 = False
                        cnt += 1
                    elif start_idx > 0:
                        start_idx -= 1
                    else:
                        collector += np.zeros((1, self.word2vec.vector_size)).flatten()
                        flag_1 = False
                while flag_2:
                    if end_idx < len(tokens) and end_idx != pos_in_window and tokens[end_idx] in word2vec:
                        collector += (self.word2vec.get_vector(tokens[end_idx]) - self.mean) / self.std 
                        flag_2 = False
                        cnt += 1
                    elif end_idx < len(tokens) - 1:
                        end_idx += 1
                    else:
                        collector += np.zeros((1, self.word2vec.vector_size)).flatten()
                        flag_2 = False
                if cnt == 0:
                    avg_emg = np.zeros((1, self.word2vec.vector_size)).flatten()
                else:
                    avg_emg = collector / cnt
                embeddings.append(avg_emg)
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [50]:
dev = TwitterDatasetAverage(dev_data, "text", "emotion", word2vec)

In [ ]:
for d in dev:
    print(d["feature"].shape)

In [51]:
batch_size = 1024
num_workers = 4

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [54]:
vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 1e-3
num_epochs = 10

model_avg = Classifier(300, 150, num_classes)# Твоя модель
model_avg = model_avg.to(device)
criterion = nn.CrossEntropyLoss() # Твой лосс
optimizer = torch.optim.Adam(model_avg.parameters(), lr=lr) # Твой оптимайзер
best_metric = np.inf
for e in range(num_epochs):
    training(model_avg, optimizer, criterion, train_loader, e, device)
    log = testing(model_avg, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model_avg.state_dict(), "model_avg.pt")
        best_metric = log["Test Loss"]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



{'Test Loss': 0.5267561454772949, 'Test Acc': 0.73516796875}



{'Test Loss': 0.5202978081703186, 'Test Acc': 0.74140625}



{'Test Loss': 0.5165551508665085, 'Test Acc': 0.743765625}



{'Test Loss': 0.5133076157569886, 'Test Acc': 0.74495703125}


...


{'Test Loss': 0.5107417509555817, 'Test Acc': 0.74680078125}



{'Test Loss': 0.5107376096248627, 'Test Acc': 0.74751171875}



{'Test Loss': 0.5088879888057709, 'Test Acc': 0.747671875}



{'Test Loss': 0.5094560567140579, 'Test Acc': 0.74774609375}



{'Test Loss': 0.5087251286506653, 'Test Acc': 0.74809375}


In [55]:
test_loader = DataLoader(
    TwitterDatasetAverage(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model_avg.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



{'Test Loss': 0.5079940152815736, 'Test Acc': 0.7494820786741214}


In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [58]:
class TwitterDatasetTfidf(Dataset):  
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec, tfidf):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec
        self.tfidf = tfidf

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)
        self._tfidf()

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])
        tfidf_emdeddings = self.tfidf.transform([text]).toarray().flatten()
        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens) + tfidf_emdeddings

        return {"feature": embeddings, "target": label}

    def _tfidf(self):
        text = [self.data[self.feature_column][item] for item in 
                range(len(self.data[self.feature_column]))]
        self.tfidf.fit_transform(text)

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        tokens = self.tokenizer.tokenize(text)
        tokens = [w for w in tokens if all(c not in string.punctuation for c in w) and len(w) > 3]
        tokens = [w for w in tokens if w in word2vec]
        return tokens

    def get_embeddings_(self, tokens):
        # Получи эмбеддинги слов и усредни их
        embeddings = [(self.word2vec.get_vector(token) - self.mean) / self.std for token in (tokens)]
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [59]:
vectorizer = TfidfVectorizer(max_features=300)
dev = TwitterDatasetTfidf(dev_data, "text", "emotion", word2vec, vectorizer)

batch_size = 1024
num_workers = 4

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
#dev.tfidf.get_feature_names()
for d in dev:
    print(d["feature"].shape)

In [ ]:
dev.tfidf.transform(["freedom comes at the cost"]).toarray().flatten()

In [ ]:
dict(zip(dev.tfidf.get_feature_names(), list(dev.tfidf.idf_)))

In [ ]:
dev.tfidf.get_feature_names()

In [60]:
vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 1e-3
num_epochs = 5

model = Classifier(300, 150, num_classes)# Твоя модель
model_avg = model.to(device)
criterion = nn.CrossEntropyLoss() # Твой лосс
optimizer = torch.optim.Adam(model.parameters(), lr=lr) # Твой оптимайзер
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model_tfidf.pt")
        best_metric = log["Test Loss"]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


KeyboardInterrupt: ignored

In [ ]:
test_loader = DataLoader(
    TwitterDatasetTfidf(test_data, "text", "emotion", word2vec, vectorizer), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model_tfidf.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

In [ ]:
class TwitterDatasetTfidf_(Dataset):  
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec, tfidf):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec
        self.tfidf = tfidf

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)
        self._tfidf()
        self.dictionary = dict(zip(self.tfidf.get_feature_names(), list(self.tfidf.idf_)))

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])
        tfidf_emdeddings = self.tfidf.transform([text]).toarray().flatten()
        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens) + tfidf_emdeddings

        return {"feature": embeddings, "target": label}

    def _tfidf(self):
        text = [self.data[self.feature_column][item] for item in 
                range(len(self.data[self.feature_column]))]
        self.tfidf.fit_transform(text)

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        tokens = self.tokenizer.tokenize(text)
        tokens = [w for w in tokens if all(c not in string.punctuation for c in w) and len(w) > 3]
        tokens = [w for w in tokens if w in word2vec]
        return tokens

    def get_embeddings_(self, tokens):
        # Получи эмбеддинги слов и усредни их
        embeddings = [(self.dictionary.get(token, 1) * self.word2vec.get_vector(token) - self.mean) / self.std for token in (tokens)]
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [ ]:
vectorizer = TfidfVectorizer(max_features=300)
dev = TwitterDatasetTfidf_(dev_data, "text", "emotion", word2vec, vectorizer)

batch_size = 1024
num_workers = 4

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

In [ ]:
vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 1e-2
num_epochs = 1

model = Classifier(300, 150, num_classes)# Твоя модель
model_avg = model.to(device)
criterion = nn.CrossEntropyLoss() # Твой лосс
optimizer = torch.optim.Adam(model.parameters(), lr=lr) # Твой оптимайзер
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model_tfidf.pt")
        best_metric = log["Test Loss"]

In [ ]:
log

In [ ]:
test_loader = DataLoader(
    TwitterDatasetTfidf_(test_data, "text", "emotion", word2vec, vectorizer), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model_tfidf.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))